# Post-hoc Repacking Analysis for H2O Molecule

This notebook demonstrates three Pauli grouping strategies:
1. **Baseline**: Sorted insertion grouping
2. **Greedy Repacking**: Optimize grouping from scratch for minimum variance
3. **Post-hoc Repacking**: Add paulis from previous groups if they're diagonal under existing circuits

We compare these methods on both simulated and real IBM hardware data.

## Setup and Imports

In [1]:
import pickle
import numpy as np
import openfermion as of
import cirq
from collections import Counter

from grouping import MeasurementGroups, sorted_insertion_grouping
from repacking import greedy_repacking, posthoc_repacking
from measurement import create_measurement_setups
from estimation import estimate_from_groups
from cache import GroupingCache

## Load H2O Hamiltonian

Load the water molecule Hamiltonian (14 qubits, 1620 Pauli terms) and compute exact HF energy.

In [2]:
# Load molecular data
mol_data = of.chem.MolecularData(filename='monomer_eqb.hdf5')
n_electrons = mol_data.n_electrons

# Convert to qubit Hamiltonian
hamiltonian_fermion = of.jordan_wigner(
    of.get_fermion_operator(mol_data.get_molecular_hamiltonian())
)
hamiltonian = of.qubit_operator_to_pauli_sum(hamiltonian_fermion)

# Setup qubits
qubits = cirq.LineQubit.range(14)

# Compute exact HF energy
hf_energy_exact = 0.0
for pauli_string in hamiltonian:
    coeff = pauli_string.coefficient
    exp_value = 1.0
    for qubit in qubits:
        if qubit in pauli_string.qubits:
            gate = pauli_string[qubit]
            qubit_idx = qubit.x
            if qubit_idx < n_electrons:
                if gate == cirq.Z:
                    exp_value *= -1
                elif gate in [cirq.X, cirq.Y]:
                    exp_value = 0.0
                    break
            else:
                if gate in [cirq.X, cirq.Y]:
                    exp_value = 0.0
                    break
    hf_energy_exact += np.real(coeff * exp_value)

print(f"Hamiltonian: {len(hamiltonian)} Pauli terms on {len(qubits)} qubits")
print(f"Exact HF energy: {hf_energy_exact:.6f} Ha")

Hamiltonian: 1620 Pauli terms on 14 qubits
Exact HF energy: -75.679017 Ha


## Baseline: Sorted Insertion Grouping

Group Pauli terms using sorted insertion - the standard greedy algorithm that groups commuting terms.

In [3]:
# Perform sorted insertion grouping
baseline_groups = sorted_insertion_grouping(hamiltonian)
baseline_setups = create_measurement_setups(baseline_groups.groups, qubits)

# Statistics
num_groups_baseline = len(baseline_groups.groups)
num_paulis_baseline = sum(len(g) for g in baseline_groups.groups)
avg_paulis_per_group_baseline = num_paulis_baseline / num_groups_baseline

print(f"Baseline Grouping Statistics:")
print(f"  Number of groups: {num_groups_baseline}")
print(f"  Total paulis: {num_paulis_baseline}")
print(f"  Average paulis per group: {avg_paulis_per_group_baseline:.1f}")
print(f"  Min paulis in a group: {min(len(g) for g in baseline_groups.groups)}")
print(f"  Max paulis in a group: {max(len(g) for g in baseline_groups.groups)}")

Baseline Grouping Statistics:
  Number of groups: 65
  Total paulis: 1619
  Average paulis per group: 24.9
  Min paulis in a group: 2
  Max paulis in a group: 105


## Greedy Repacking

Repack strings to minimize expected variance.  This does a second greedy pass scoring strings according to c_i^2/N_i

In [4]:
# Perform greedy repacking
greedy_groups = greedy_repacking(baseline_groups, verbose=False)
greedy_setups = create_measurement_setups(greedy_groups.groups, qubits)

# Statistics
num_groups_greedy = len(greedy_groups.groups)
num_paulis_greedy = sum(len(g) for g in greedy_groups.groups)
avg_paulis_per_group_greedy = num_paulis_greedy / num_groups_greedy

print(f"Greedy Repacking Statistics:")
print(f"  Number of groups: {num_groups_greedy}")
print(f"  Total paulis: {num_paulis_greedy}")
print(f"  Average paulis per group: {avg_paulis_per_group_greedy:.1f}")
print(f"  Min paulis in a group: {min(len(g) for g in greedy_groups.groups)}")
print(f"  Max paulis in a group: {max(len(g) for g in greedy_groups.groups)}")

# Comparison to baseline
paulis_added_greedy = num_paulis_greedy - num_paulis_baseline
percent_increase_greedy = (paulis_added_greedy / num_paulis_baseline) * 100

print(f"\nComparison to Baseline:")
print(f"  Paulis added: {paulis_added_greedy} ({percent_increase_greedy:.0f}% increase)")
print(f"  Groups changed: {num_groups_baseline} (same)")

Greedy Repacking Statistics:
  Number of groups: 65
  Total paulis: 5057
  Average paulis per group: 77.8
  Min paulis in a group: 33
  Max paulis in a group: 105

Comparison to Baseline:
  Paulis added: 3438 (212% increase)
  Groups changed: 65 (same)


## Post-hoc Repacking

Add paulis from previous groups if they commute with current group AND are diagonal under the group's circuit. 

In [5]:
# Perform post-hoc repacking
posthoc_groups = posthoc_repacking(
    baseline_groups,
    baseline_setups,
    qubits,
    verbose=False
)
posthoc_setups = create_measurement_setups(posthoc_groups.groups, qubits)

# Statistics
num_groups_posthoc = len(posthoc_groups.groups)
num_paulis_posthoc = sum(len(g) for g in posthoc_groups.groups)
avg_paulis_per_group_posthoc = num_paulis_posthoc / num_groups_posthoc

print(f"Post-hoc Repacking Statistics:")
print(f"  Number of groups: {num_groups_posthoc}")
print(f"  Total paulis: {num_paulis_posthoc}")
print(f"  Average paulis per group: {avg_paulis_per_group_posthoc:.1f}")
print(f"  Min paulis in a group: {min(len(g) for g in posthoc_groups.groups)}")
print(f"  Max paulis in a group: {max(len(g) for g in posthoc_groups.groups)}")

# Comparison to baseline
paulis_added_posthoc = num_paulis_posthoc - num_paulis_baseline
percent_increase_posthoc = (paulis_added_posthoc / num_paulis_baseline) * 100

print(f"\nComparison to Baseline:")
print(f"  Paulis added: {paulis_added_posthoc} ({percent_increase_posthoc:.0f}% increase)")
print(f"  Groups changed: {num_groups_baseline} (same)")

# Circuit reuse analysis
circuits_reused = 0
circuits_conjugated = 0
circuits_exact = 0
circuits_regenerated = 0

for i in range(len(baseline_setups)):
    baseline_circuit = baseline_setups[i].basis_rotation
    posthoc_circuit = posthoc_setups[i].basis_rotation
    
    if baseline_circuit == posthoc_circuit:
        circuits_exact += 1
        circuits_reused += 1
    elif len(baseline_circuit) == len(posthoc_circuit):
        # Check if they're the same up to qubit ordering
        circuits_conjugated += 1
        circuits_reused += 1
    else:
        circuits_regenerated += 1

print(f"\nCircuit Reuse:")
print(f"  Exact matches: {circuits_exact}")
print(f"  Conjugated: {circuits_conjugated}")
print(f"  Regenerated: {circuits_regenerated}")
print(f"  Total reused: {circuits_reused}/{len(baseline_setups)} ({circuits_reused/len(baseline_setups)*100:.0f}%)")

Post-hoc Repacking Statistics:
  Number of groups: 65
  Total paulis: 4825
  Average paulis per group: 74.2
  Min paulis in a group: 31
  Max paulis in a group: 105

Comparison to Baseline:
  Paulis added: 3206 (198% increase)
  Groups changed: 65 (same)

Circuit Reuse:
  Exact matches: 6
  Conjugated: 2
  Regenerated: 57
  Total reused: 8/65 (12%)


## Simulation Results

Run simulations with the Hartree-Fock state to compare variance reduction across all three methods.

In [6]:
# Create HF state
hf_state = [1] * n_electrons + [0] * (len(qubits) - n_electrons)
shots_per_group = 100000

print(f"Running simulations with {shots_per_group:,} shots per group...\n")

# Generate measurement counts for each method
def simulate_measurements(groups, setups, state, shots):
    """Simulate measurements for all groups."""
    all_counts = []
    for group_idx, (group, setup) in enumerate(zip(groups.groups, setups)):
        circuit = setup.basis_rotation
        
        # Apply circuit to state and get probabilities
        simulator = cirq.Simulator()
        initial_state_cirq = cirq.Circuit([cirq.X(qubits[i]) for i, bit in enumerate(state) if bit == 1])
        full_circuit = initial_state_cirq + circuit
        result = simulator.simulate(full_circuit)
        
        # Sample from the final state
        sampler = cirq.Simulator()
        samples = sampler.run(full_circuit + cirq.measure(*qubits), repetitions=shots)
        
        # Convert to counts dictionary
        counts = Counter()
        for measurement in samples.measurements.values():
            for bitstring in measurement:
                key = ''.join(str(int(b)) for b in bitstring)
                counts[key] += 1
        
        all_counts.append(counts)
    
    return all_counts

# Simulate all three methods
print("Simulating baseline...")
baseline_counts = simulate_measurements(baseline_groups, baseline_setups, hf_state, shots_per_group)

print("Simulating greedy repacking...")
greedy_counts = simulate_measurements(greedy_groups, greedy_setups, hf_state, shots_per_group)

print("Simulating post-hoc repacking...")
posthoc_counts = simulate_measurements(posthoc_groups, posthoc_setups, hf_state, shots_per_group)

print("\nDone!")

Running simulations with 100,000 shots per group...

Simulating baseline...
Simulating greedy repacking...
Simulating post-hoc repacking...

Done!


In [7]:
# Estimate energies from simulated counts
shots_list = [shots_per_group] * num_groups_baseline

baseline_sim_results = estimate_from_groups(
    baseline_groups,
    baseline_setups,
    baseline_counts,
    shots_list,
    qubits
)

greedy_sim_results = estimate_from_groups(
    greedy_groups,
    greedy_setups,
    greedy_counts,
    shots_list,
    qubits
)

posthoc_sim_results = estimate_from_groups(
    posthoc_groups,
    posthoc_setups,
    posthoc_counts,
    shots_list,
    qubits
)

# Display results
print("="*80)
print("Simulation Results (HF State, 100k shots/group)")
print("="*80)
print(f"\nExact HF energy: {hf_energy_exact:.6f} Ha\n")

print(f"Baseline:")
print(f"  Energy: {baseline_sim_results.energy:.6f} ± {baseline_sim_results.energy_std():.6f} Ha")
print(f"  Error: {abs(baseline_sim_results.energy - hf_energy_exact):.6f} Ha")
print(f"  Variance: {baseline_sim_results.energy_variance:.6e}")

print(f"\nGreedy Repacking:")
print(f"  Energy: {greedy_sim_results.energy:.6f} ± {greedy_sim_results.energy_std():.6f} Ha")
print(f"  Error: {abs(greedy_sim_results.energy - hf_energy_exact):.6f} Ha")
print(f"  Variance: {greedy_sim_results.energy_variance:.6e}")
print(f"  Variance reduction: {baseline_sim_results.energy_variance / greedy_sim_results.energy_variance:.2f}x")

print(f"\nPost-hoc Repacking:")
print(f"  Energy: {posthoc_sim_results.energy:.6f} ± {posthoc_sim_results.energy_std():.6f} Ha")
print(f"  Error: {abs(posthoc_sim_results.energy - hf_energy_exact):.6f} Ha")
print(f"  Variance: {posthoc_sim_results.energy_variance:.6e}")
print(f"  Variance reduction: {baseline_sim_results.energy_variance / posthoc_sim_results.energy_variance:.2f}x")

print("\n" + "="*80)

Simulation Results (HF State, 100k shots/group)

Exact HF energy: -75.679017 Ha

Baseline:
  Energy: -75.680290 ± 0.004028 Ha
  Error: 0.001273 Ha
  Variance: 1.622725e-05

Greedy Repacking:
  Energy: -75.679137 ± 0.001768 Ha
  Error: 0.000120 Ha
  Variance: 3.126856e-06
  Variance reduction: 5.19x

Post-hoc Repacking:
  Energy: -75.680305 ± 0.001941 Ha
  Error: 0.001288 Ha
  Variance: 3.768386e-06
  Variance reduction: 4.31x



## Hardware Results

In [11]:
# Load September 25 hardware data
with open('all_counts_fez_sep25.pkl', 'rb') as f:
    hardware_counts_sep = pickle.load(f)

shots_per_group_hw = [sum(counts.values()) for counts in hardware_counts_sep]
total_shots = sum(shots_per_group_hw)

print(f"September 2025 Hardware Data:")
print(f"  Groups: {len(hardware_counts_sep)}")
print(f"  Total shots: {total_shots:,}")
print(f"  Shots per group: {shots_per_group_hw[0]:,}\n")

# Analyze with baseline and post-hoc
baseline_hw_sep = estimate_from_groups(
    baseline_groups,
    baseline_setups,
    hardware_counts_sep,
    shots_per_group_hw,
    qubits
)

posthoc_hw_sep = estimate_from_groups(
    posthoc_groups,
    posthoc_setups,
    hardware_counts_sep,
    shots_per_group_hw,
    qubits
)

variance_reduction_sep = baseline_hw_sep.energy_variance / posthoc_hw_sep.energy_variance
std_reduction_sep = baseline_hw_sep.energy_std() / posthoc_hw_sep.energy_std()

print("="*80)
print("September 25 Hardware Results")
print("="*80)
print(f"\nExact HF energy: {hf_energy_exact:.6f} Ha\n")

print(f"Baseline:")
print(f"  Energy: {baseline_hw_sep.energy:.6f} ± {baseline_hw_sep.energy_std():.6f} Ha")
print(f"  Error from exact: {abs(baseline_hw_sep.energy - hf_energy_exact):.6f} Ha")
print(f"  Variance: {baseline_hw_sep.energy_variance:.6e}")

print(f"\nPost-hoc Repacking:")
print(f"  Energy: {posthoc_hw_sep.energy:.6f} ± {posthoc_hw_sep.energy_std():.6f} Ha")
print(f"  Error from exact: {abs(posthoc_hw_sep.energy - hf_energy_exact):.6f} Ha")
print(f"  Variance: {posthoc_hw_sep.energy_variance:.6e}")

print(f"\nVariance reduction: {variance_reduction_sep:.2f}x ({(1-1/variance_reduction_sep)*100:.1f}% reduction)")
print(f"Std dev reduction: {std_reduction_sep:.2f}x")
print("="*80)

September 2025 Hardware Data:
  Groups: 65
  Total shots: 3,250,000
  Shots per group: 50,000

September 25 Hardware Results

Exact HF energy: -75.679017 Ha

Baseline:
  Energy: -73.551561 ± 0.006340 Ha
  Error from exact: 2.127456 Ha
  Variance: 4.019046e-05

Post-hoc Repacking:
  Energy: -67.995741 ± 0.011542 Ha
  Error from exact: 7.683276 Ha
  Variance: 1.332203e-04

Variance reduction: 0.30x (-231.5% reduction)
Std dev reduction: 0.55x


In [ ]:
# Load October 22 hardware data
with open('all_counts_fez_oct22_3.pkl', 'rb') as f:
    hardware_counts_oct = pickle.load(f)

print(f"October 22 Hardware Data:")
print(f"  Groups: {len(hardware_counts_oct)}")
print(f"  Total shots: {sum(sum(counts.values()) for counts in hardware_counts_oct):,}")
print(f"  Shots per group: {sum(hardware_counts_oct[0].values()):,}\n")

# Analyze with baseline and post-hoc
baseline_hw_oct = estimate_from_groups(
    baseline_groups,
    baseline_setups,
    hardware_counts_oct,
    shots_per_group_hw,
    qubits
)

posthoc_hw_oct = estimate_from_groups(
    posthoc_groups,
    posthoc_setups,
    hardware_counts_oct,
    shots_per_group_hw,
    qubits
)

variance_reduction_oct = baseline_hw_oct.energy_variance / posthoc_hw_oct.energy_variance
std_reduction_oct = baseline_hw_oct.energy_std() / posthoc_hw_oct.energy_std()

print("="*80)
print("October 22 Hardware Results")
print("="*80)
print(f"\nExact HF energy: {hf_energy_exact:.6f} Ha\n")

print(f"Baseline:")
print(f"  Energy: {baseline_hw_oct.energy:.6f} ± {baseline_hw_oct.energy_std():.6f} Ha")
print(f"  Error from exact: {abs(baseline_hw_oct.energy - hf_energy_exact):.6f} Ha")
print(f"  Variance: {baseline_hw_oct.energy_variance:.6e}")

print(f"\nPost-hoc Repacking:")
print(f"  Energy: {posthoc_hw_oct.energy:.6f} ± {posthoc_hw_oct.energy_std():.6f} Ha")
print(f"  Error from exact: {abs(posthoc_hw_oct.energy - hf_energy_exact):.6f} Ha")
print(f"  Variance: {posthoc_hw_oct.energy_variance:.6e}")

print(f"\nVariance reduction: {variance_reduction_oct:.2f}x ({(1-1/variance_reduction_oct)*100:.1f}% reduction)")
print(f"Std dev reduction: {std_reduction_oct:.2f}x")
print("="*80)

October 22 Hardware Data:
  Groups: 65
  Total shots: 3,250,000
  Shots per group: 50,000



## Summary Comparison Table

Side-by-side comparison of all results.

In [10]:
import pandas as pd

summary_data = [
    {
        'Dataset': 'Simulation',
        'Method': 'Baseline',
        'Energy (Ha)': f"{baseline_sim_results.energy:.6f}",
        'Std Dev (Ha)': f"{baseline_sim_results.energy_std():.6f}",
        'Variance': f"{baseline_sim_results.energy_variance:.2e}",
        'Var. Reduction': '1.00x',
        'Circuit Reuse': '100%'
    },
    {
        'Dataset': 'Simulation',
        'Method': 'Greedy',
        'Energy (Ha)': f"{greedy_sim_results.energy:.6f}",
        'Std Dev (Ha)': f"{greedy_sim_results.energy_std():.6f}",
        'Variance': f"{greedy_sim_results.energy_variance:.2e}",
        'Var. Reduction': f"{baseline_sim_results.energy_variance / greedy_sim_results.energy_variance:.2f}x",
        'Circuit Reuse': 'N/A'
    },
    {
        'Dataset': 'Simulation',
        'Method': 'Post-hoc',
        'Energy (Ha)': f"{posthoc_sim_results.energy:.6f}",
        'Std Dev (Ha)': f"{posthoc_sim_results.energy_std():.6f}",
        'Variance': f"{posthoc_sim_results.energy_variance:.2e}",
        'Var. Reduction': f"{baseline_sim_results.energy_variance / posthoc_sim_results.energy_variance:.2f}x",
        'Circuit Reuse': '100%'
    },
    {
        'Dataset': 'HW Sep 2025',
        'Method': 'Baseline',
        'Energy (Ha)': f"{baseline_hw_sep.energy:.6f}",
        'Std Dev (Ha)': f"{baseline_hw_sep.energy_std():.6f}",
        'Variance': f"{baseline_hw_sep.energy_variance:.2e}",
        'Var. Reduction': '1.00x',
        'Circuit Reuse': '100%'
    },
    {
        'Dataset': 'HW Sep 2025',
        'Method': 'Post-hoc',
        'Energy (Ha)': f"{posthoc_hw_sep.energy:.6f}",
        'Std Dev (Ha)': f"{posthoc_hw_sep.energy_std():.6f}",
        'Variance': f"{posthoc_hw_sep.energy_variance:.2e}",
        'Var. Reduction': f"{variance_reduction_sep:.2f}x",
        'Circuit Reuse': '100%'
    },
    {
        'Dataset': 'HW Oct 2022',
        'Method': 'Baseline',
        'Energy (Ha)': f"{baseline_hw_oct.energy:.6f}",
        'Std Dev (Ha)': f"{baseline_hw_oct.energy_std():.6f}",
        'Variance': f"{baseline_hw_oct.energy_variance:.2e}",
        'Var. Reduction': '1.00x',
        'Circuit Reuse': '100%'
    },
    {
        'Dataset': 'HW Oct 2022',
        'Method': 'Post-hoc',
        'Energy (Ha)': f"{posthoc_hw_oct.energy:.6f}",
        'Std Dev (Ha)': f"{posthoc_hw_oct.energy_std():.6f}",
        'Variance': f"{posthoc_hw_oct.energy_variance:.2e}",
        'Var. Reduction': f"{variance_reduction_oct:.2f}x",
        'Circuit Reuse': '100%'
    }
]

df = pd.DataFrame(summary_data)
print("\n" + "="*100)
print("SUMMARY: All Results")
print("="*100)
print(f"Exact HF Energy: {hf_energy_exact:.6f} Ha")
print("="*100)
print(df.to_string(index=False))
print("="*100)


SUMMARY: All Results
Exact HF Energy: -75.679017 Ha
    Dataset   Method Energy (Ha) Std Dev (Ha) Variance Var. Reduction Circuit Reuse
 Simulation Baseline  -75.680290     0.004028 1.62e-05          1.00x          100%
 Simulation   Greedy  -75.679137     0.001768 3.13e-06          5.19x           N/A
 Simulation Post-hoc  -75.680305     0.001941 3.77e-06          4.31x          100%
HW Sep 2025 Baseline  -73.551561     0.006340 4.02e-05          1.00x          100%
HW Sep 2025 Post-hoc  -67.995741     0.011542 1.33e-04          0.30x          100%
HW Oct 2022 Baseline  -52.837890     0.062171 3.87e-03          1.00x          100%
HW Oct 2022 Post-hoc  -49.897433     0.016396 2.69e-04         14.38x          100%
